# Maladie Cardiaque 

# Importation des bliotheques de bases

In [1]:
import pandas as pd
import numpy as np
import missingno as mno
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn import metrics

# Chargement des données

In [2]:
Heart_Dataset = pd.read_csv("heart.dat", sep="\s+", names=["age", "sex", "chest_pain", "blood_pressure", "serum_cholestoral", "fasting_blood_sugar", "electrocardiographic", "max_heart_rate", "induced_angina", "oldpeak", "slope", "major vessels", "thal", "Label"])

In [ ]:
Heart_Dataset.head()

ici nous presentons notre jeu de données

# Analyse statistique descriptive

### Profilage des donnees

In [ ]:
Heart_Dataset.info()

In [ ]:
Heart_Dataset.isnull().sum()

In [ ]:
mno.matrix(Heart_Dataset)

Nous constatons ici d'apres les trois approches utilisées que le jeu de donnees ne presente pas de valeurs nulles.

In [4]:
Heart_Dataset.describe()

,age,sex,chest_pain,blood_pressure,serum_cholestoral,fasting_blood_sugar,electrocardiographic,max_heart_rate,induced_angina,oldpeak,slope,major vessels,thal,Label
count,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.00000,270.000000,270.000000,270.000000,270.000000
mean,54.433333,0.677778,3.174074,131.344444,249.659259,0.148148,1.022222,149.677778,0.329630,1.05000,1.585185,0.670370,4.696296,1.444444
std,9.109067,0.468195,0.950090,17.861608,51.686237,0.355906,0.997891,23.165717,0.470952,1.14521,0.614390,0.943896,1.940659,0.497827
min,29.000000,0.000000,1.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.00000,1.000000,0.000000,3.000000,1.000000
25%,48.000000,0.000000,3.000000,120.000000,213.000000,0.000000,0.000000,133.000000,0.000000,0.00000,1.000000,0.000000,3.000000,1.000000
50%,55.000000,1.000000,3.000000,130.000000,245.000000,0.000000,2.000000,153.500000,0.000000,0.80000,2.000000,0.000000,3.000000,1.000000
75%,61.000000,1.000000,4.000000,140.000000,280.000000,0.000000,2.000000,166.000000,1.000000,1.60000,2.000000,1.000000,7.000000,2.000000
max,77.000000,1.000000,4.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.20000,3.000000,3.000000,7.000000,2.000000


#### Commentaire

Pour les colonnes age, blood_pressure, serum_cholestoral et max_heart_rate on note une dispersion assez elevee des donnees par rapport à la moyenne.
Par rapport aux quartiles, la repartition des donnees est assez differentes selon les colonnes. En effet, pour les colonnes age, blood_pressure, serum_cholestoral, on note plus une repartition assez homogene des donnees sur q1, q2 et q3 avec une des proportions croissantes en passant de q1, q2 et q3. Pour d'autres colonnes comme max_heart_rate, la tendance de la repartition est differentes des colonnes citées ci-dessus.
S'agissant de la moyenne, Il est notable de remarquer l'heterogeneité des echelles. En effet, la moyenne des colonnes du dataset sont tres differentes : Ceci peut etre problematique durant la phase d'apprentissage ; ainsi, un travail de normalisation sera fait pour contourner ce probleme

# Correlation des donnees

In [ ]:
Heart_Dataset.corr()

In [3]:
corr = Heart_Dataset.corr()
corr.style.background_gradient(cmap='coolwarm')

,age,sex,chest_pain,blood_pressure,serum_cholestoral,fasting_blood_sugar,electrocardiographic,max_heart_rate,induced_angina,oldpeak,slope,major vessels,thal,Label
age,1.000000,-0.094401,0.096920,0.273053,0.220056,0.123458,0.128171,-0.402215,0.098297,0.194234,0.159774,0.356081,0.106100,0.212322
sex,-0.094401,1.000000,0.034636,-0.062693,-0.201647,0.042140,0.039253,-0.076101,0.180022,0.097412,0.050545,0.086830,0.391046,0.297721
chest_pain,0.096920,0.034636,1.000000,-0.043196,0.090465,-0.098537,0.074325,-0.317682,0.353160,0.167244,0.136900,0.225890,0.262659,0.417436
blood_pressure,0.273053,-0.062693,-0.043196,1.000000,0.173019,0.155681,0.116157,-0.039136,0.082793,0.222800,0.142472,0.085697,0.132045,0.155383
serum_cholestoral,0.220056,-0.201647,0.090465,0.173019,1.000000,0.025186,0.167652,-0.018739,0.078243,0.027709,-0.005755,0.126541,0.028836,0.118021
fasting_blood_sugar,0.123458,0.042140,-0.098537,0.155681,0.025186,1.000000,0.053499,0.022494,-0.004107,-0.025538,0.044076,0.123774,0.049237,-0.016319
electrocardiographic,0.128171,0.039253,0.074325,0.116157,0.167652,0.053499,1.000000,-0.074628,0.095098,0.120034,0.160614,0.114368,0.007337,0.182091
max_heart_rate,-0.402215,-0.076101,-0.317682,-0.039136,-0.018739,0.022494,-0.074628,1.000000,-0.380719,-0.349045,-0.386847,-0.265333,-0.253397,-0.418514
induced_angina,0.098297,0.180022,0.353160,0.082793,0.078243,-0.004107,0.095098,-0.380719,1.000000,0.274672,0.255908,0.153347,0.321449,0.419303
oldpeak,0.194234,0.097412,0.167244,0.222800,0.027709,-0.025538,0.120034,-0.349045,0.274672,1.000000,0.609712,0.255005,0.324333,0.417967


Les colonnes ont des correlations souvent tres faibles entre elles. Ceci teimoigne d'une absence de redondance d'informations tres probable sur l'ensemble de donnees.

# Data visualisation

Analyse univariée

In [ ]:
print(Heart_Dataset.Label.value_counts())
sns.countplot(y="Label", hue="Label", data=Heart_Dataset)

On constate que l'absence de maladie est sensiblement plus importante que sa presence dans notre jeu de donnees, Cependant, nous ne somme pas dans une disproportion car les valeurs sont plus ou moins proches 

In [ ]:
print(Heart_Dataset.sex.value_counts())
sns.countplot(y="sex", hue="sex", data=Heart_Dataset)

Ce graphe montre que la proportion de sexe maculin est plus important que celle de sexe feminin.Alors là par contre on constate une difference importante entre les deux proportions

In [ ]:
print(Heart_Dataset.chest_pain.value_counts())
sns.countplot(x="chest_pain", hue="chest_pain", data=Heart_Dataset)

Les douleurs thoraciques de niveau 4 sont tres representees dans l'ensemble de donnees. Elles sont suvie par le niveau 3, ensuite par le niveau 2 et 1 en suivant la meme ordre.

In [ ]:
print(Heart_Dataset.electrocardiographic.value_counts())
sns.countplot(y="electrocardiographic", hue="electrocardiographic", data=Heart_Dataset)

ici nous constatons que les individus présentants des riques cardiaques ont un electrocardiogramme trés faibles (ce qui est normal)

In [ ]:
print(Heart_Dataset.thal.value_counts())
sns.countplot(y="thal", hue="thal", data=Heart_Dataset)

Dans cette reprsentation graphique, on constate la proportion d'individus présentant des douleurs thoraxique est plus importante chez les personnes présentantes des risques cardiaques

In [ ]:
sns.distplot(Heart_Dataset.age, rug=True, hist=True)

La plus part des patient ont un age compris entre 40 - 70 annees. Les personnes qui pourraient etre atteintes par la maladie seraient tres probablement d'une age avancé

In [ ]:
sns.distplot(Heart_Dataset.max_heart_rate, rug=True, hist=True)

la frequence cardiaque normale devrant envoisiner 60 à 80 pulsations par minute chez les personnes saines, on constate ici que les frequences cardiaques peuvent atteindre jusqu'a 170 pulsations de densité chez les personnes malades 

In [ ]:
sns.distplot(Heart_Dataset.blood_pressure, rug=True, hist=True)

la pression arterielle pourrait etre tres élevée chez les personnes malades

In [ ]:
sns.distplot(Heart_Dataset.serum_cholestoral, rug=True, hist=True)

Analyse bivariée detaillé dans le rapport

In [ ]:
sns.boxplot(x='Label',y='age', data=Heart_Dataset)

In [ ]:
plt.figure(figsize=(25,5))
plt.subplot(1,4,1)
sns.boxplot(x='Label',y='oldpeak', data=Heart_Dataset)
plt.subplot(1,4,2)
sns.boxplot(x='Label',y='max_heart_rate', data=Heart_Dataset)
plt.subplot(1,4,3)
sns.boxplot(x='Label',y='serum_cholestoral', data=Heart_Dataset)
plt.subplot(1,4,4)
sns.boxplot(x='Label',y='blood_pressure', data=Heart_Dataset);

On voit que la plus des variables presente des valeurs aberrentes.

In [ ]:
# Répartition graphique des exemples en fonction de toutes les combinaisons de variables 2 à 2
sns.pairplot(Heart_Dataset, hue='Label', corner = True, palette = ['orange', 'blue'], height=4)

ici nous avons globalement une distribution gaussienne sur l'ensemble des variables. On voit qu'il y'a des variables qui nous permettent d'observer deux modes sur les labels et d'autres variables qui observents trois modes 

# Apprentissage du modele predictif

### Entrainement du modèle de prediction

## Strategie 1: Procédure d'estimation du SVM sur un noyau lineaire

In [ ]:
#decoupage de mon jeu de données en deux parties
X = Heart_Dataset.drop(['Label'], axis=1)
y = Heart_Dataset['Label']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [ ]:
#normalisation de mon jeu de données
from sklearn import preprocessing
scandler = preprocessing.StandardScaler().fit(X_train)
X_train_Std = scandler.transform(X_train)
X_test_std = scandler.transform(X_test)

In [ ]:
### Procédure d'estimation d'un SVM linéaire pour différentes valeurs de C
### -> GridSearchCV

parameters = {'kernel': ['linear'], 'C': np.logspace(-1, 1, 3)}
score = 'accuracy' #'recall'

print("# Réglage des hyper-paramètres en fonction de %s" % score)
clf = GridSearchCV(SVC(), parameters, scoring=score)
clf.fit(X_train_Std, y_train)

print("Résultats de validation sur l'ensemble des paramètres:")
print(clf, clf.cv_results_)        
print("Meilleurs paramètres:")
print(clf.best_params_)

In [ ]:
### Résultats de test pour les meilleurs paramètres

print("Rapport de classification sur l'ensemble de test:")
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))

Ici, nous avons une trés mauvaise precision de notre modele cela prouve qu'il est totalement impossible d'utiliser un noyau linaire pour classer nos données

## Strategie 2: Procédure d'estimation du SVM sur un noyau gaussienne

In [ ]:
# choisir 6 valeurs pour C, entre 1e-2 et 1e3
C_range = np.logspace(-2, 3, 6)

# choisir 4 valeurs pour gamma, entre 1e-2 et 10
gamma_range = np.logspace(-2, 1, 4)

# grille de paramètres
param_grid = {'C': C_range, 'gamma': gamma_range}

# critère de sélection du meilleur modèle
score = 'accuracy'

# initialiser une recherche sur grille
grid = model_selection.GridSearchCV(svm.SVC(kernel='rbf', class_weight={1:35 , 2:1}), 
                                    param_grid, 
                                    cv=5, # 5 folds de validation croisée  
                                    scoring=score,
                                    refit = True, 
                                    verbose = 3)

# faire tourner la recherche sur grille
grid.fit(X_train_Std, y_train)

# afficher les paramètres optimaux
print("The optimal parameters are {} with a score of {:.2f}".format(grid.best_params_, grid.best_score_))

In [ ]:
# prédire sur le jeu de test avec le modèle optimisé
print("Rapport de classification sur l'ensemble de test:")
y_true, y_pred = y_test, clf.predict(X_test_std)
print(classification_report(y_true, y_pred))

ici gràce à la distribution gaussienne dans la globalité de nos variables, nous constatons des performances de modele 
Cette nouvelle strategie presente ainsi des resultats bien meilleurs que la precedente. En effet, les performances
 du modele sur le jeu de donnees test donne des resultats plus satisfaisants sur la precision, le rappel et
 f1-score. Ce qui teimoigne d'une capacite de generalisation que le modele a.